In [1]:
import spatialproteomics as sp
from skimage.io import imread
import matplotlib.pyplot as plt
from scipy.signal import medfilt2d
import pandas as pd


In [2]:
image = imread("/dss/dsshome1/00/ge65tib2/Spatialproteomics/reg002_X01_Y01_Z01.tif")
image.shape

(10, 9060, 7524)

In [3]:
markers = ['DAPI', 'Ki67', 'Keratin5', 'CD68', 'Podoplanin', 'SMA', 'CD107a', 'CLDN17',
       'CD34', 'HLA-DR']
sp_object = sp.load_image_data(image, channel_coords=markers)



In [4]:
sp_object

<xarray.Dataset> Size: 682MB
Dimensions:   (channels: 10, y: 9060, x: 7524)
Coordinates:
  * channels  (channels) <U10 400B 'DAPI' 'Ki67' 'Keratin5' ... 'CD34' 'HLA-DR'
  * y         (y) int64 72kB 0 1 2 3 4 5 6 ... 9054 9055 9056 9057 9058 9059
  * x         (x) int64 60kB 0 1 2 3 4 5 6 ... 7518 7519 7520 7521 7522 7523
Data variables:
    _image    (channels, y, x) uint8 682MB 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0

In [5]:
ds_cellpose = sp_object.tl.cellpose(channel="DAPI", model_type='nuclei', flow_threshold=0.6)



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.4 
platform:       	linux 
python version: 	3.10.18 
torch version:  	2.7.1+cu126! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 




** TORCH CUDA version installed and working. **
>>>> using GPU (CUDA)
>>>> loading model /dss/dsshome1/00/ge65tib2/.cellpose/models/cpsam


In [6]:
ds_cellpose = ds_cellpose.pp.add_quantification(func="intensity_mean")


In [7]:
ds_cellpose

<xarray.Dataset> Size: 824MB
Dimensions:        (channels: 10, y: 9060, x: 7524, cells: 58005, features: 2)
Coordinates:
  * channels       (channels) <U10 400B 'DAPI' 'Ki67' ... 'CD34' 'HLA-DR'
  * y              (y) int64 72kB 0 1 2 3 4 5 ... 9054 9055 9056 9057 9058 9059
  * x              (x) int64 60kB 0 1 2 3 4 5 ... 7518 7519 7520 7521 7522 7523
  * cells          (cells) int64 464kB 1 2 3 4 5 ... 58002 58003 58004 58005
  * features       (features) <U10 80B 'centroid-0' 'centroid-1'
Data variables:
    _image         (channels, y, x) uint8 682MB 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    _segmentation  (y, x) uint16 136MB 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    _obs           (cells, features) float64 928kB 92.66 2.87e+03 ... 4.024e+03
    _intensity     (cells, channels) float64 5MB 8.15 0.008333 ... 0.6821

In [8]:
# putting the expression matrix into an anndata object
adata = ds_cellpose.tl.convert_to_anndata(
    expression_matrix_key="_intensity",
)

adata
adata.write("/dss/dsshome1/00/ge65tib2/Scimap/fresh_frozen_reg002_cellpose.h5ad")
